### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [62]:
def return_metrics(df1, df2):
    '''
    input: Pandas Dataframe df1, df2
    output: some metrics related to the questions below
    '''
    num_movies = len(df1['movie'].unique())
    num_ratings = len(df2['rating'])
    uniq_users = len(df2['user_id'].unique())
    missing_ratings = int(df2['rating'].isnull().sum())
    average_rating = round(df2['rating'].mean(), 0)
    min_rating = df2['rating'].min()
    max_rating = df2['rating'].max()
    
    unique_genres = []
    for _ in df1['genre']:
        try:
            unique_genres.extend(_.split('|'))
        except:
            pass
    unique_genres = set(unique_genres)
    
    print(num_movies, num_ratings, len(unique_genres), uniq_users, missing_ratings, average_rating, min_rating, max_rating)
    
    return unique_genres

In [63]:
unique_genres = return_metrics(movies, reviews)

35416 863866 28 67353 0 7.0 0 10


In [64]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 35416,
'The number of ratings in the dataset': 863866,
'The number of different genres': 28,
'The number of unique users in the dataset': 67353,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.0,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [67]:
# cell for work
movies['date'] = movies['movie'].str[-5:-1]

In [68]:
# and another
def dummy_century(value):
    if value[:2] == year:
        return 1
    else:
        return 0
    
for year in ['18', '19', '20']:
    movies[str(year) + "00's"] = movies['date'].apply(dummy_century)

In [72]:
# and a few more below, which you can use as necessary
def splitting_genres(value):
    try:
        if value.find(genre) > -1:
            return 1
        else:
            return 0
    except:
        return 0
    
for genre in unique_genres:
    movies[genre] = movies['genre'].apply(splitting_genres)

In [75]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Documentary,Romance,Film-Noir,...,Adult,Western,War,Music,Animation,Talk-Show,History,Biography,Horror,Musical
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [77]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [80]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 21:00:50
1,2,0208092,5,1586466072,2020-04-09 21:01:12
2,2,0358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [81]:
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')